# Install requirements through pip

In [1]:
!pip install --user tensorflow==2.3.1
!pip install --user nltk==3.5
!pip install --user colorama==0.4.3
!pip install --user numpy==1.18.5
!pip install --user scikit_learn==0.23.2
!pip install --user Flask==1.1.2
!pip install -U mapbox
!pip install --user requests

# Imports

In [17]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras


import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import pandas as pd
import pickle
import random
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Mels\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

# Load  patterns & responses (intents.json)

In [76]:
with open('intents.json') as file:
    data = json.load(file)
    
ws = []
cs = []
ds = []
ignore_ws = ['?']
# Check all intents
for i in data['intents']:
    # Check for pattern in each intent
    for p in i['patterns']:
        # get token from all words
        w = nltk.word_tokenize(p)
        # add to our word list
        ws.extend(w)
        ds.append((w, i['tag']))
        # add to class list
        if i['tag'] not in cs:
            cs.append(i['tag'])
# remove duplicates
ws = [stemmer.stem(w.lower()) for w in ws if w not in ignore_ws]
ws = sorted(list(set(ws)))
# sort classes
cs = sorted(list(set(cs)))
# Docs = combination between patterns and intents
print (len(ds), "docs")
# classes = intents
print (len(cs), "classes", cs)
# words = all ws, vocabulary
print (len(ws), "unique words", ws)

21 docs
6 classes ['goodbye', 'greeting', 'help_route', 'options', 'search_building_by_name', 'set_curr_loc']
42 unique words [',', 'be', 'build', 'bye', 'can', 'chat', 'could', 'cur', 'day', 'do', 'find', 'for', 'go', 'good', 'goodby', 'hello', 'help', 'hey', 'hi', 'hol', 'how', 'i', 'is', 'lat', 'loc', 'me', 'my', 'next', 'nic', 'provid', 'room', 'search', 'see', 'show', 'the', 'til', 'tim', 'to', 'want', 'way', 'what', 'you']


# Create train- and testdata

In [19]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word - create base word, in attempt to represent related words
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])

C:\Users\Mels\AppData\Local\Temp/ipykernel_2604/1377439408.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [20]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [21]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
# Fit the model
model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

D:\Anaconda\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/200
5/5 [==============================] - ETA: 1s - loss: 1.9792 - accuracy: 0.20 - ETA: 0s - loss: 1.9508 - accuracy: 0.23 - 1s 17ms/step - loss: 1.9508 - accuracy: 0.2381
Epoch 2/200
5/5 [==============================] - ETA: 0s - loss: 2.0368 - accuracy: 0.0000e+ - 0s 4ms/step - loss: 1.7770 - accuracy: 0.1905
Epoch 3/200
5/5 [==============================] - ETA: 0s - loss: 1.6382 - accuracy: 0.20 - 0s 3ms/step - loss: 1.7482 - accuracy: 0.2857
Epoch 4/200
5/5 [==============================] - ETA: 0s - loss: 1.6990 - accuracy: 0.20 - 0s 4ms/step - loss: 1.6456 - accuracy: 0.2857
Epoch 5/200
5/5 [==============================] - ETA: 0s - loss: 1.7566 - accuracy: 0.20 - 0s 3ms/step - loss: 1.5843 - accuracy: 0.2857
Epoch 6/200
5/5 [==============================] - ETA: 0s - loss: 1.7819 - accuracy: 0.20 - 0s 4ms/step - loss: 1.5350 - accuracy: 0.4762
Epoch 7/200
5/5 [==============================] - ETA: 0s - loss: 1.2790 - accuracy: 0.60 - 0s 4ms/step - loss: 1.4855

5/5 [==============================] - ETA: 0s - loss: 0.1288 - accuracy: 1.00 - 0s 4ms/step - loss: 0.1242 - accuracy: 1.0000
Epoch 60/200
5/5 [==============================] - ETA: 0s - loss: 0.0929 - accuracy: 1.00 - 0s 4ms/step - loss: 0.1917 - accuracy: 0.9524
Epoch 61/200
5/5 [==============================] - ETA: 0s - loss: 0.0272 - accuracy: 1.00 - 0s 4ms/step - loss: 0.0872 - accuracy: 1.0000
Epoch 62/200
5/5 [==============================] - ETA: 0s - loss: 0.3136 - accuracy: 0.80 - 0s 4ms/step - loss: 0.1427 - accuracy: 0.9524
Epoch 63/200
5/5 [==============================] - ETA: 0s - loss: 0.0534 - accuracy: 1.00 - 0s 4ms/step - loss: 0.1366 - accuracy: 0.9524
Epoch 64/200
5/5 [==============================] - ETA: 0s - loss: 0.1516 - accuracy: 0.80 - 0s 3ms/step - loss: 0.1336 - accuracy: 0.9524
Epoch 65/200
5/5 [==============================] - ETA: 0s - loss: 0.1312 - accuracy: 1.00 - 0s 4ms/step - loss: 0.0704 - accuracy: 1.0000
Epoch 66/200
5/5 [===============

5/5 [==============================] - ETA: 0s - loss: 0.0205 - accuracy: 1.00 - 0s 3ms/step - loss: 0.0174 - accuracy: 1.0000
Epoch 118/200
5/5 [==============================] - ETA: 0s - loss: 2.0098e-05 - accuracy: 1.00 - 0s 3ms/step - loss: 0.0255 - accuracy: 1.0000
Epoch 119/200
5/5 [==============================] - ETA: 0s - loss: 0.0322 - accuracy: 1.00 - 0s 3ms/step - loss: 0.0460 - accuracy: 1.0000
Epoch 120/200
5/5 [==============================] - ETA: 0s - loss: 0.0035 - accuracy: 1.00 - 0s 3ms/step - loss: 0.0262 - accuracy: 1.0000
Epoch 121/200
5/5 [==============================] - ETA: 0s - loss: 0.0110 - accuracy: 1.00 - 0s 3ms/step - loss: 0.0123 - accuracy: 1.0000
Epoch 122/200
5/5 [==============================] - ETA: 0s - loss: 0.1170 - accuracy: 1.00 - 0s 3ms/step - loss: 0.0324 - accuracy: 1.0000
Epoch 123/200
5/5 [==============================] - ETA: 0s - loss: 0.0013 - accuracy: 1.00 - 0s 3ms/step - loss: 0.0632 - accuracy: 0.9524
Epoch 124/200
5/5 [====

Epoch 175/200
5/5 [==============================] - ETA: 0s - loss: 0.0036 - accuracy: 1.00 - 0s 4ms/step - loss: 0.0105 - accuracy: 1.0000
Epoch 176/200
5/5 [==============================] - ETA: 0s - loss: 0.0146 - accuracy: 1.00 - 0s 4ms/step - loss: 0.0050 - accuracy: 1.0000
Epoch 177/200
5/5 [==============================] - ETA: 0s - loss: 4.2935e-05 - accuracy: 1.00 - 0s 6ms/step - loss: 0.0275 - accuracy: 1.0000
Epoch 178/200
5/5 [==============================] - ETA: 0s - loss: 0.0013 - accuracy: 1.00 - 0s 4ms/step - loss: 0.0412 - accuracy: 1.0000
Epoch 179/200
5/5 [==============================] - ETA: 0s - loss: 0.0021 - accuracy: 1.00 - 0s 4ms/step - loss: 0.0331 - accuracy: 1.0000
Epoch 180/200
5/5 [==============================] - ETA: 0s - loss: 0.0028 - accuracy: 1.00 - 0s 5ms/step - loss: 0.0015 - accuracy: 1.0000
Epoch 181/200
5/5 [==============================] - ETA: 0s - loss: 0.0055 - accuracy: 1.00 - 0s 4ms/step - loss: 0.0356 - accuracy: 1.0000
Epoch 182

In [40]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

In [41]:
def predict_class(sentence):
    ERROR_THRESHOLD = 0.25
    
    # generate probabilities from the model
    input_data = pd.DataFrame([bow(sentence, words)], dtype=float, index=['input'])
    results = model.predict([input_data])[0]
    # filter out predictions below a threshold, and provide intent index
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], str(r[1])))
    # return tuple of intent and probability
    
    return return_list

In [66]:
def getResponse(ints, intents_json):
    tag = ints[0][0]
    print(tag)
    list_of_intents = intents_json['intents']
    
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(text):
    ints = predict_class(text)
    res = getResponse(ints,data)
    return res

In [77]:
import tkinter
from tkinter import *


def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)
    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))

        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')

        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)
        
        
base = Tk()
base.title("Hello")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)


#Create Chat windowChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial", background="#dddddd")
#EntryBox.bind("<Return>", send)


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=130, y=401, height=40, width=265)
SendButton.place(x=6, y=401, height=40)

In [78]:
base.mainloop()

found in bag: hello
greeting
found in bag: how
found in bag: can
found in bag: you
found in bag: help
found in bag: me
options


# Giving real directions (unimplemented)

In [ ]:
import googlemaps
from mapbox import Directions
def calc_route(bld):
    dicbuilding = buildings.get(bld)
    if not dicbuilding is None:
        response = service.directions((global_currloc, dicbuilding),'mapbox.walking')
        print(response.geojson())
    